In [25]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, r2_score
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from scipy import stats

from xgboost import XGBRegressor

In [26]:
df = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/Final_Presentation/Processed data/finaldata_w_xy_and_dummies.csv')

In [27]:
#df.drop('Unnamed: 0', axis=1, inplace=True)
df.columns

Index(['Unnamed: 0', 'gameId', 'playId', 'x', 'y', 'nflId', 'event',
       'displayName', 'frameId', 'targetNflId', 'Opp_Dist', 'closestOpp_Id',
       'Team_Dist', 'closestTeam_Id', 'QB_Dist', 'closestQB_Id', 'FootDist',
       'quarter', 'down', 'yardsToGo', 'defendersInTheBox',
       'numberOfPassRushers', 'absoluteYardlineNumber', 'passResult',
       'penalty', 'offensePlayResult', 'epa', 'playResult', 'isDefensivePI',
       'height', 'weight', 'position_CB', 'position_DB', 'position_DE',
       'position_DT', 'position_FB', 'position_HB', 'position_K',
       'position_QB', 'position_RB', 'position_TE', 'position_WR',
       'route_ANGLE', 'route_CORNER', 'route_CROSS', 'route_FLAT', 'route_GO',
       'route_HITCH', 'route_IN', 'route_OUT', 'route_POST', 'route_SCREEN',
       'route_SLANT', 'route_WHEEL', 'route_undefined'],
      dtype='object')

In [28]:
df = df[['offensePlayResult','quarter','down','yardsToGo','Opp_Dist','Team_Dist','QB_Dist','FootDist',\
           'defendersInTheBox','numberOfPassRushers','height','weight','position_CB', 'position_DB', \
           'position_DE', 'position_DT','position_FB', 'position_HB', 'position_K', 'position_QB', \
           'position_RB', 'position_TE', 'position_WR', 'route_ANGLE','route_CORNER', 'route_CROSS', \
           'route_FLAT', 'route_GO', 'route_HITCH', 'route_IN', 'route_OUT', 'route_POST', \
           'route_SCREEN', 'route_SLANT','route_WHEEL',]]

In [29]:
# format heigh to number in inches
height_dict = {'5-10':70,'6-5':77, '6-4':76, '6-3':75, '6-2':74, '6-1':73, '6-0':72, '5-11':71, \
               '5-9':69, '5-8':68, '5-7':67, '5-6':66,'6-7':79,'6-6':78,'64':64,'65':65,'66':66,'67':67,'68':68, \
              '69':69,'70':70,'71':71,'72':72,'73':73,'74':74,'75':75,'76':76,'77':77,'78':78,'79':79,'80':80}
df['height'] = df['height'].map(height_dict) 

In [30]:
# Split data into train, validation, test
X = df.drop(['offensePlayResult',], axis=1)
y = df['offensePlayResult']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, shuffle=True, random_state=42)

feature_names = X.columns
X_train = pd.DataFrame(data=X_train, columns=feature_names)
X_val = pd.DataFrame(data=X_val, columns=feature_names)

print('Train observations: ',len(X_train))
print('Validation observations: ',len(X_val))
print('Test observations: ',len(X_test))

Train observations:  9921
Validation observations:  3308
Test observations:  4410


In [124]:
param_dist = {'n_estimators': [stats.randint(150, 500)],
              'learning_rate': [stats.uniform(0.01, 0.07)],
              'subsample': [stats.uniform(0.3, 0.7)],
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': [stats.uniform(0.5, 0.45)],
              'min_child_weight': [1, 2, 3]
             }

params = {'min_child_weight':[4,5],
          'subsample':[i/10.0 for i in range(6,11)],
            'colsample_bytree':[i/10.0 for i in range(6,11)],
          'max_depth': [2,3,4]}

reg_CV = XGBRegressor()

In [31]:
reg_CV_run = GridSearchCV(reg_CV, params)
reg_CV_run.fit(X_train, y_train)

NameError: name 'reg_CV' is not defined

In [126]:
reg_CV_run.best_params_

{'colsample_bytree': 0.9,
 'max_depth': 2,
 'min_child_weight': 4,
 'subsample': 1.0}

In [32]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

finalized_CV = XGBRegressor(colsample_bytree= 0.9,
 max_depth= 2,
 min_child_weight= 4,
 subsample= 1.0).fit(X_train, y_train)

CV_pred = finalized_CV.predict(X_test)

In [33]:
r2_score(y_test, CV_pred)

0.14740424817310316

In [34]:
mean_absolute_error(y_test, CV_pred)

6.694917046084425

In [35]:
mean_squared_error(y_test, CV_pred)

92.91704423920368

In [36]:
def mean_absolute_percentage_error(y_true, y_pred): 


    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [37]:
mean_absolute_percentage_error(y_test, CV_pred)

inf